# Process and grade LLM responses

In [1]:
import pandas as pd
from collections import Counter
from typing import NamedTuple
import numpy as np

In [2]:
class InputFiles(NamedTuple):
    responses: str
    taxonomy_scores: str
    taxon_counts: str
    train_test_split: str

class OutputFiles(NamedTuple):
    results_with_ums: str

class Env(NamedTuple):
    input_files: InputFiles
    output_files: OutputFiles
    num_phrasings: int
    phrasings: list[str]
    query_fields: list[str]
    seed: int
    only_valid_absences: bool

if "snakemake" in globals():
    env = Env(
        input_files=snakemake.input,
        output_files=snakemake.output,
        num_phrasings=len(snakemake.params.phrasings),
        phrasings=snakemake.params.phrasings,
        query_fields=snakemake.params.query_fields,
        seed=snakemake.params.seed,
        only_valid_absences=snakemake.params.validate_absences
    )
else: # Fill in parameters manually for testing outside of snakemake
    import os
    ROOT = os.path.expanduser("~/biodiversity-llms")
    LLM = "gpt-4-1106-preview"

    env = Env(
        input_files = InputFiles(
            responses=f"../../results/{LLM}/all-shuffled.tsv",
            taxonomy_scores=ROOT + "/tdwg2023/taxonomy/results/kpfg_scores.tsv",
            taxon_counts=ROOT + "/tdwg2023/taxonomy/results/taxon-counts.tsv",
            train_test_split="../../results/input/train_test_split.tsv"
        ),
        output_files = OutputFiles(
            results_with_ums=f"../../results/{LLM}/results-with-ums.tsv"
        ),
        num_phrasings=6,
        phrasings=[
            "Can species {genus} {specificepithet} be found in {county}, {stateprovince}, {country}?",
            "Is it possible to encounter species {genus} {specificepithet} in {county}, {stateprovince}, {country}?",
            "Is there a presence of species {genus} {specificepithet} within {county}, {stateprovince}, {country}?",
            "Does {county}, {stateprovince}, {country} harbor species {genus} {specificepithet}?",
            "Is species {genus} {specificepithet} present in {county}, {stateprovince}, {country}?",
            "Can one observe species {genus} {specificepithet} in {county}, {stateprovince}, {country}?"
        ],
        query_fields=[
            "kingdom",
            "phylum",
            "family",
            "genus",
            "specificepithet",
            "country",
            "stateprovince",
            "county"
        ],
        seed=69847,
        only_valid_absences=True
    )

def nest(level, strings):
    separator = "\n" + "  " * level + "- "
    return separator + separator.join([str(s) for s in strings])

def quote(strings):
    return [f'"{s}"' for s in strings]

print("Job parameters:")
print(f"- Responses to analyze: {env.input_files.responses}")
print(f"- Query phrasings (count: {env.num_phrasings}):{nest(2, quote(env.phrasings))}")
print(f"- Query fields:{nest(2, quote(env.query_fields))}")


Job parameters:
- Responses to analyze: ../../results/gpt-4-1106-preview/all-shuffled.tsv
- Query phrasings (count: 6):
    - "Can species {genus} {specificepithet} be found in {county}, {stateprovince}, {country}?"
    - "Is it possible to encounter species {genus} {specificepithet} in {county}, {stateprovince}, {country}?"
    - "Is there a presence of species {genus} {specificepithet} within {county}, {stateprovince}, {country}?"
    - "Does {county}, {stateprovince}, {country} harbor species {genus} {specificepithet}?"
    - "Is species {genus} {specificepithet} present in {county}, {stateprovince}, {country}?"
    - "Can one observe species {genus} {specificepithet} in {county}, {stateprovince}, {country}?"
- Query fields:
    - "kingdom"
    - "phylum"
    - "family"
    - "genus"
    - "specificepithet"
    - "country"
    - "stateprovince"
    - "county"


In [3]:
def count_item(values, item):
    counts = dict(Counter(values).most_common())
    return counts[item] if item in counts else 0

def make_id(df):
    return df.apply(lambda r: hash("".join([str(v) for v in r.values]).lower()), axis=1)

def get_results(responses, train_test_split):
    train_test_split = pd.read_csv(open(train_test_split), sep="\t", index_col=0)
    df = pd.read_csv(open(responses), sep="\t").merge(train_test_split, on=env.query_fields)

    df["phrasing"] = df["question number"].astype(int) % env.num_phrasings
    df["query id"] = make_id(df[env.query_fields])

    df["response id"] = make_id(df[["query id", "phrasing"]])
    df = df.groupby("response id").head(1) # Drop responses for repeated questions

    df["scores"] = df["responses"].apply(lambda r: count_item(r.lower().split(), "yes"))
    df["yesnos"] = df["responses"].apply(lambda r: count_item(r.lower().split(), "yes") + count_item(r.lower().split(), "no"))
    df["abstains"] = 10 - df["yesnos"]

    df["prediction"] = df["scores"].apply(lambda x: -1 if x == 0 else 1)
    df["correct"] = df["prediction"] * df["target"]

    return df

UNUSED_FIELDS = ["query"]
res = get_results(env.input_files.responses, env.input_files.train_test_split)\
    .drop(columns=UNUSED_FIELDS)

print(f"{len(res) / env.num_phrasings:,.0f} records")
print(f"{len(res):,.0f} queries (#records x #phrasings)")
res.head(1)

FileNotFoundError: [Errno 2] No such file or directory: '../../results/gpt-4-1106-preview/all-shuffled.tsv'

In [ ]:
def find_optimal_decision_threshold():
    def decision(x):
        return x.astype(int) * 2 - 1

    scores = res["scores"]
    thresholds = np.arange(scores.min(), scores.max(), (scores.max() - scores.min()) / 1000)
    accuracies = np.array(list(map(lambda t: (decision(scores >= t) == res["target"]).mean(), thresholds)))
    best_accuracy_threshold = accuracies.argmax()
    print(f"Best accuracy threshold: between {thresholds[best_accuracy_threshold - 1]:.1g} - {thresholds[best_accuracy_threshold]:.1g}")
find_optimal_decision_threshold()

Best accuracy threshold: between 0 - 0.01


Query-level statistics

In [6]:
pd.read_csv(open(env.input_files.taxonomy_scores, "r"), sep="\t")

,subject rank,taxon,num_correct,num_response
0,family,acaulosporaceae,31.0,36.0
1,family,acroporidae,40.0,40.0
2,family,agaricaceae,36.0,36.0
3,family,amphiuridae,30.0,30.0
4,family,apidae,40.0,40.0
...,...,...,...,...
2281,phylum,porifera,10.0,10.0
2282,phylum,rhodophyta,7.0,7.0
2283,phylum,spermatophyta,0.0,10.0
2284,phylum,tracheophyta,10.0,10.0


In [4]:
kpfg_scores = pd.read_csv(open(env.input_files.taxonomy_scores, "r"), sep="\t")
kpfg_scores["accuracy"] = (1 + kpfg_scores["num_correct"]) / (2 + kpfg_scores["num_response"])
kpfg_scores = kpfg_scores.set_index(["subject rank", "taxon"])
average_kpfg_scores = kpfg_scores.groupby("subject rank").sum().apply(lambda r: r["num_correct"] / r["num_response"], axis=1)
average_kpfg_responses = kpfg_scores.groupby("subject rank")["num_response"].mean()

record_counts_by_taxon = pd.read_csv(open(env.input_files.taxon_counts, "r"), sep="\t").set_index(["kingdom", "phylum", "family"])

In [11]:
phrasing_avg_pred = res.groupby("query id")["prediction"].mean()
phrasing_var_score = res.groupby("query id")["scores"].var()

In [12]:
def show_accuracies(df, field, title, remap_values={}, format="{:,.2%}"):
    df = pd.DataFrame(
        (df.groupby(field)["correct"].mean() * .5 + .5)
        .rename("Response accuracy")
    ).transpose()
    df.columns.name = title
    df.rename(columns=remap_values, inplace=True)

    display(df.style.format(format))

show_accuracies(res, "phrasing", "Phrasing")
show_accuracies(res, "target", "Actual presence", remap_values={-1: "Absent", 1: "Present"})
show_accuracies(res, "prediction", "Predicted presence", remap_values={-1: "Absent", 1: "Present"})
show_accuracies(res, "kingdom", "Kingdom")

Phrasing,0,1,2,3,4,5
Response accuracy,64.30%,64.11%,64.48%,64.20%,63.88%,63.73%


Actual presence,Absent,Present
Response accuracy,86.31%,43.56%


Predicted presence,Absent,Present
Response accuracy,58.61%,77.47%


Kingdom,animalia,plantae
Response accuracy,63.79%,64.54%


In [13]:
def get_acc_by_field(d, field, prior_counts=1) -> pd.Series:
    def fill_blanks(series):
        return series.reindex(d[field].unique(), fill_value=0)
    
    subset = d["train"]
    num_correct = prior_counts + fill_blanks(d[subset * (d["correct"] == 1)].groupby(field).size())
    num_incorrect = prior_counts + fill_blanks(d[subset * (d["correct"] == -1)].groupby(field).size())

    return num_correct / (num_correct + num_incorrect)

df = res[res["phrasing"] == 0]

acc_by_rank = pd.Series({rank: get_acc_by_field(df, rank) for rank in ["kingdom", "phylum", "family"]})
acc_by_country = get_acc_by_field(df, "country")
acc_by_stateprovince = get_acc_by_field(df, "stateprovince")

def get_ums(instance):
    # TODO: condition um3 on prediction?
    um1_present = instance["scores"]
    um1_absent = 1 if um1_present == 0 else 0
    taxon_record_counts = record_counts_by_taxon.loc[instance["kingdom"], instance["phylum"], instance["family"]]

    # Positively oriented, i.e. higher values = more certainty
    return pd.Series({
        # Scores
        "um1_present_score": um1_present,
        "um1_absent_score": um1_absent,
        "um1_either": um1_present + um1_absent,
        
        # Abstains
        "um2_percent_abstains": 1.0 - instance["abstains"] / 10.0,

        # Performance by field values
        # Note: defaults to 0.5 if no data (TODO)
        "um3_accuracy_by_kingdom": acc_by_rank["kingdom"][instance["kingdom"]],
        "um3_accuracy_by_phylum": acc_by_rank["phylum"][instance["phylum"]],
        "um3_accuracy_by_family": acc_by_rank["family"][instance["family"]],
        "um3_accuracy_by_country": acc_by_country[instance["country"]],
        "um3_accuracy_by_stateprovince": acc_by_stateprovince[instance["stateprovince"]],

        # Sensitivity to phrasing
        "um4_phrasing_agreement": phrasing_avg_pred[instance["query id"]] * -instance["prediction"],
        "um4_phrasing_score_var": phrasing_var_score[instance["query id"]],

        # iDigBio record counts by taxonomic ranks
        "um5_record_count_by_kingdom": taxon_record_counts["kingdomCount"],
        "um5_record_count_by_phylum": taxon_record_counts["phylumCount"],
        "um5_record_count_by_family": taxon_record_counts["familyCount"],
        # "um5_record_count_by_country": country_record_count,
        # "um5_record_count_by_stateprovince": stateprovince_record_count,

        # Accuracy on taxonomy questions
        "um6_taxqa_accuracy_by_phylum": kpfg_scores["accuracy"]["phylum"].get(instance["phylum"].lower(), average_kpfg_scores["phylum"]),
        "um6_taxqa_accuracy_by_family": kpfg_scores["accuracy"]["family"].get(instance["family"].lower(), average_kpfg_scores["family"]),
        "um6_taxqa_accuracy_by_genus": kpfg_scores["accuracy"]["genus"].get(instance["genus"].lower(), average_kpfg_scores["genus"]),

        # Number of yes-no responses to taxonomy questions
        "um7_taxqa_responses_by_phylum": kpfg_scores["num_response"]["phylum"].get(instance["phylum"].lower(), average_kpfg_responses["phylum"]) / 10,
        "um7_taxqa_responses_by_family": kpfg_scores["num_response"]["family"].get(instance["family"].lower(), average_kpfg_responses["family"]) / 40,
        "um7_taxqa_responses_by_genus": kpfg_scores["num_response"]["genus"].get(instance["genus"].lower(), average_kpfg_responses["genus"]) / 50,
    })

full_df = pd.concat([df.apply(lambda row: get_ums(row), axis=1), df], axis=1)
full_df.to_csv(env.output_files.results_with_ums)